In [1]:
from math import floor
from random import random  # For testing


# Making a simple model

In [2]:
test_denominations = [100, 50, 20, 10, 5, 1]
test_value = 37

result_dictionary = dict()
for denomination in test_denominations:
    denomination_count = floor(test_value / denomination)
    if denomination_count > 0:
        result_dictionary[denomination] = denomination_count
        test_value = test_value - denomination * denomination_count


# Writing the simple model into a function

In [3]:
def change_breakdown(cash):
    denominations = [100, 50, 20, 10, 5, 1, 0.25, 0.10, 0.01]
    result_dictionary = dict()
    for denomination in denominations:
        denomination_count = floor(cash / denomination)
        if denomination_count > 0:
            result_dictionary[denomination] = denomination_count
            cash = cash - denomination * denomination_count
    return result_dictionary

# Test a few values
test_list = [37, 0, 289.52]
for value in test_list:
    print(change_breakdown(value))

# Write a function for testing that recombines the change
def test_output(dictionary):
    products_list = [denomination * count for denomination, count in dictionary.items()]
    sum_products = sum(products_list)
    print(f"{dictionary} recombines to ${sum_products}")

# Test a few values
for value in test_list:
    print(f"${value}: ", end='')
    test_output(change_breakdown(value))


{20: 1, 10: 1, 5: 1, 1: 2}
{}
{100: 2, 50: 1, 20: 1, 10: 1, 5: 1, 1: 4, 0.25: 2, 0.01: 1}
$37: {20: 1, 10: 1, 5: 1, 1: 2} recombines to $37
$0: {} recombines to $0
$289.52: {100: 2, 50: 1, 20: 1, 10: 1, 5: 1, 1: 4, 0.25: 2, 0.01: 1} recombines to $289.51


# Making change machine class

At this point I used about 30 minutes of the allowed hour, so I made the `change_breakdown` function into a method of a `ChangeMachine` class. This was to improve future expandability (adding new methods) and portability (creating a module).

In [4]:
class ChangeMachine:
    def __init__(self):
        self.denominations = [100, 50, 20, 10, 5, 1, 0.25, 0.10, 0.05, 0.01]
        
    def _validate_input(self, input_value):
        """Restricts values to floats and integers."""
        input_is_float = type(input_value) == float
        input_is_int = type(input_value) == int
        if input_is_float or input_is_int:
            return input_value >= 0
        else:
            return False
    
    def add_denomination(self, denomination):
        """Lets user add less-common denominations like $2 bills and 50-cent coins."""
        if self._validate_input(denomination) == True:
            denomination_set = set(self.denominations)
            denomination_set.add(denomination)
            new_denomination_list = list(denomination_set)
            new_denomination_list = reversed(sorted(new_denomination_list))
            self.denominations = list(new_denomination_list)
    
    def remove_denomination(self, denomination):
        if denomination in self.denominations:
            self.denominations.remove(denomination)
    
    def change_breakdown(self, cash):
        result_dictionary = dict()
        for denomination in self.denominations:
            denomination_count = floor(cash / denomination)
            if denomination_count > 0:
                result_dictionary[denomination] = denomination_count
                cash = cash - denomination * denomination_count
                cash = round(cash, 2)
        return result_dictionary
    
    def transaction(self, cost=1, cash=1):
        if self._validate_input(cost) and self._validate_input(cash):
            if cash < cost:
                print("Not enough money to cover transaction.")
            else:
                change = cash - cost
                change = round(change, 2)
                breakdown = self.change_breakdown(change)
                print("Change is ${0}. The breakdown is:".format(change))
                for denomination, count in breakdown.items():
                    print("${0}: {1}".format(denomination, count))
        else:
            print("Invalid transaction inputs.")


# Testing `ChangeMachine`

In [5]:
change_machine = ChangeMachine()

print("Testing with 5 random transactions")
dollarify = lambda value: round(value * 100, 2)
for _ in range(5):
    test_cost = dollarify(random())
    test_cash = dollarify(random())
    print("Paying for ${0:0.2f} with ${1:0.2f}".format(test_cost, test_cash))
    change_machine.transaction(test_cost, test_cash)


print("\nTesting edge cases")
change_machine.transaction(0, 1.01)
change_machine.transaction(0, 1000.01)
change_machine.transaction(1.01, 0)
change_machine.transaction(-1, 0)


Testing with 5 random transactions
Paying for $81.17 with $36.38
Not enough money to cover transaction.
Paying for $18.11 with $45.95
Change is $27.84. The breakdown is:
$20: 1
$5: 1
$1: 2
$0.25: 3
$0.05: 1
$0.01: 4
Paying for $17.14 with $20.53
Change is $3.39. The breakdown is:
$1: 3
$0.25: 1
$0.1: 1
$0.01: 4
Paying for $11.16 with $36.36
Change is $25.2. The breakdown is:
$20: 1
$5: 1
$0.1: 2
Paying for $41.38 with $3.63
Not enough money to cover transaction.

Testing edge cases
Change is $1.01. The breakdown is:
$1: 1
$0.01: 1
Change is $1000.01. The breakdown is:
$100: 10
$0.01: 1
Not enough money to cover transaction.
Invalid transaction inputs.
